# CALFEWS: California Food-Energy-Water System simulation model

This notebook will rerun all simulations and recreate all figures from the following manuscript:

Zeff, H.B., Herman, J.D., Hamilton, A.L., Malek, K., Cohen, J.S., Medellin-Azuara, J., Reed, P.M., and G.W. Characklis. (2020). "Paper title." *Journal Name*. (In preparation).

Prior to running this notebook, follow the instructions in "README.md" to download the model and data, and install all Python dependencies.

## Run validation 

Make sure you do not already have a folder "results/baseline_wy2017" from a previous run. If so, you will get an error.

#### Override parameter file with version in "runtime_params_validation.ini"

In [ ]:
! cp runtime_params_validation.ini runtime_params.ini

#### Run model. This will take ~15 minutes.

In [ ]:
! python3 -W ignore run_main_cy.py

## Run simulation 
#### Now change to simulation mode and rerun. This will take ~1.5 hours.

In [ ]:
! sed -i 's:\"validation\":\"simulation\":' runtime_params.ini
! python3 -W ignore run_main_cy.py

## Analyze & visualize data. This may take a couple of hours.

In [ ]:
import numpy as np
import pandas as pd
import h5py
import json
from datetime import datetime
import matplotlib.pyplot as plt
from itertools import compress
import os
from calfews_src import *
from calfews_src.visualizer import Visualizer
from datetime import datetime

startTime = datetime.now()

In [ ]:
#results hdf5 file location from CALFEWS simulations
flow_input_source = 'CDEC'
show_plot = True

output_folder_val = 'results/baseline_wy2017/validation/' + flow_input_source + '/'
output_folder_sim = 'results/baseline_wy2017/simulation/' + flow_input_source + '/'
output_file_val = output_folder_val + 'results_' + flow_input_source + '.hdf5'
output_file_sim = output_folder_sim + 'results_' + flow_input_source + '.hdf5'
##get model variable names
modelno_val = pd.read_pickle(output_folder_val + 'modelno0.pkl')
modelno_sim = pd.read_pickle(output_folder_sim + 'modelno0.pkl')
modelso_val = pd.read_pickle(output_folder_val + 'modelso0.pkl')
modelso_sim = pd.read_pickle(output_folder_sim + 'modelso0.pkl')

##Folder to write visualization files
fig_folder = 'results/baseline_wy2017/figures/'
sankeys_folder = 'results/baseline_wy2017/sankeys/'
os.makedirs(fig_folder, exist_ok=True)
os.makedirs(sankeys_folder, exist_ok=True)

##plot figures in python
##Set up data for validation results
validation = Visualizer(modelso_val.district_list, modelso_val.private_list, modelso_val.city_list, modelso_val.contract_list, modelso_val.waterbank_list, modelso_val.leiu_list)
validation.get_results_sensitivity_number(output_file_val, flow_input_source, 10, 1996, 1)
validation.set_figure_params()
#Set up data for scenario results
simulation = Visualizer(modelso_sim.district_list, modelso_sim.private_list, modelso_sim.city_list, modelso_sim.contract_list, modelso_sim.waterbank_list, modelso_sim.leiu_list)
simulation.get_results_sensitivity_number(output_file_sim, flow_input_source, 10, 1905, 1)
simulation.set_figure_params()


In [ ]:
##Compare Delta pumping/outflow distributions between scenarios
print('Scenario Comp')
plot_type = 'delta_pumping'
plot_name = 'extended_simulation'
simulation.scenario_compare(fig_folder, plot_type, plot_name, validation.values, show_plot)

print('Time elapsed: ', datetime.now() - startTime)

In [ ]:
#Plot district deliveries - both the physical location (for links w/GW and PMP modelling) and the 'account' - for financial risk
plot_type = 'district_water_use'
water_use_plots = ['annual', 'monthly']
for plot_name in water_use_plots:
  print('Deliveries ' + plot_name)
  ##Uses the 'scenario' files
  #simulation.make_deliveries_by_district(fig_folder, plot_type, plot_name, show_plot)
  validation.make_deliveries_by_district(fig_folder, plot_type, plot_name, flow_input_source, show_plot)
  simulation.make_deliveries_by_district(fig_folder, plot_type, plot_name, flow_input_source, show_plot)

print('Time elapsed: ', datetime.now() - startTime)

In [ ]:
##Plot snowpack/flow relationships for different watersheds
plot_type = 'state_estimation'
forecast_plots = ['publication', 'sacramento', 'sanjoaquin', 'tulare']
n_days_colorbar = 180
scatter_plot_interval = 30
range_sensitivity = 5.0 # higher number limits range of regression function plotting closer to the range of the data
for plot_name in forecast_plots:
  print('Forcasts ' + plot_name)
  validation.plot_forecasts(fig_folder, plot_type, plot_name, n_days_colorbar, scatter_plot_interval, range_sensitivity, show_plot)
    
print('Time elapsed: ', datetime.now() - startTime)

In [ ]:
#Plot 'states' for contracts, reservoirs, districts
reservoir_name = 'sanluisstate'
district_label = 'wheeler'
district_key = 'WRM'
plot_type = 'state_response'
district_private_labels = []
district_private_keys = []
print('State Response, ' + district_key)
##Uses the 'vaidation' files
validation.show_state_response(fig_folder, plot_type, reservoir_name, district_label, district_key, district_private_labels, district_private_keys, show_plot)

print('Time elapsed: ', datetime.now() - startTime)

In [ ]:
##Plot validation between model & observations
print('Model Validation')
#delta pumping variables
plot_type = 'model_validation'
plot_name = 'delta'
use_scatter = True
validation.make_validation_timeseries(fig_folder, plot_type, plot_name, show_plot, use_scatter)

print('Time elapsed: ', datetime.now() - startTime)

In [ ]:
#sierra reservoirs
plot_name = 'sierra'
use_scatter = False
num_cols = 3
validation.make_validation_timeseries(fig_folder, plot_type, plot_name, show_plot, use_scatter, num_cols)

print('Time elapsed: ', datetime.now() - startTime)

In [ ]:
#san luis reservoirs
plot_name = 'sanluis'
use_scatter = True
validation.make_validation_timeseries(fig_folder, plot_type, plot_name, show_plot, use_scatter)

print('Time elapsed: ', datetime.now() - startTime)

In [ ]:
#groundwater banks
plot_name = 'bank'
use_scatter = True
validation.make_validation_timeseries(fig_folder, plot_type, plot_name, show_plot, use_scatter)

print('Time elapsed: ', datetime.now() - startTime)

In [ ]:
#sankey diagrams - write figures for water 'flows' in each day, then write to GIF
plot_type = 'flow_diagram'
plot_name = 'tulare'
timesteps = 20000 #timesteps need to be greater than the snapshot range
snapshot_range = (14600, 14975, 100)
simulation.plot_account_flows(sankeys_folder, plot_type, plot_name, timesteps, snapshot_range)
print('Time elapsed: ', datetime.now() - startTime)

**note: this^ did create figs, gif, but crashed kernel. I will look into it. Visualizing output below

In [ ]:
simulation.make_gif(sankeys_folder + 'cali_sankey', '1946', 14600, 14975, 100)
print('Time elapsed: ', datetime.now() - startTime)

In [ ]:
from IPython.display import HTML
HTML('<img src="results/baseline_wy2017/sankeys/cali_sankey1946.gif">')
print('Time elapsed: ', datetime.now() - startTime)